In [13]:
from jina import Flow, Executor, requests, Document, DocumentArray
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('Games.csv')
df.head()

,Console,GameName,Review,Score
0,PC,Baldur's Gate 3 Early Access Review,Early Access,6
1,NS,Control: Ultimate Edition Cloud Version Review,Good,7
2,"XONE, PC, PS4",Doom Eternal: The Ancient Gods Part 1 Review,Good,7
3,"XONE, PC, PS4",Watch Dogs: Legion Review,Great,8
4,PC,Ring Of Pain Review,Great,8


In [15]:
df = df.drop_duplicates().dropna()
df.shape

(14785, 4)

In [16]:
for ind in range(5):
    print()
    print(f" Game's name : {df.iloc[ind, 1]} ".center(10, ' '))
    print(f" Console     : {df.iloc[ind,0]}".center(10,' '))
    print(f" Review      : {df.iloc[ind, -2]}".center(10,' '))
    print(f" Score       : {df.iloc[ind,-1]}".center(10,' '))
    print('-' * 80)


 Game's name : Baldur's Gate 3 Early Access Review 
 Console     : PC
 Review      : Early Access
 Score       : 6
--------------------------------------------------------------------------------

 Game's name : Control: Ultimate Edition Cloud Version Review 
 Console     : NS
 Review      : Good
 Score       : 7
--------------------------------------------------------------------------------

 Game's name : Doom Eternal: The Ancient Gods Part 1 Review 
 Console     : XONE, PC, PS4
 Review      : Good
 Score       : 7
--------------------------------------------------------------------------------

 Game's name : Watch Dogs: Legion Review 
 Console     : XONE, PC, PS4
 Review      : Great
 Score       : 8
--------------------------------------------------------------------------------

 Game's name : Ring Of Pain Review 
 Console     : PC
 Review      : Great
 Score       : 8
--------------------------------------------------------------------------------


In [17]:
docs = DocumentArray()
for ind in range(df.shape[0]):
    name = df.iloc[ind, 1]
    console = df.iloc[ind, 0]
    review = df.iloc[ind, -2]
    score = df.iloc[ind, -1]    
    doc = Document(text = name)
    doc.tags['console'] = console
    docs.append(doc)
docs

<jina.types.arrays.document.DocumentArray length=14785 at 2998280436896>

In [18]:
docs[0].json()


'{\n  "id": "cab426f7-3e20-11ec-968c-a4b1c1874569",\n  "mime_type": "text/plain",\n  "tags": {\n    "console": "PC"\n  },\n  "text": "Baldur\'s Gate 3 Early Access Review"\n}'

### EXECUTOR

In [19]:
class Clean(Executor):
    @requests
    def clean(self, docs, parameters, **kwargs):
        print(parameters)
        return DocumentArray(list(filter(lambda x : len(x.text) > 0, docs)))

In [20]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name="error_text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
    )
    .add(
        name="error_text_indexer",
        uses='jinahub://SimpleIndexer',
    )
)

In [21]:
docs = docs[:100]
docs

<jina.types.arrays.document.DocumentArray length=100 at 2998735969152>

In [22]:
with flow:
    flow.index(
      inputs=docs,
        docs = docs,
        parameters = {'name' : 'somethign'}
  )

InvalidRequirement: Parse error at "'-f https'": Expected W:(abcd...)